In [33]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import pickle

In [34]:
with open('/Users/alejandro/AnacondaProjects/train_set_new_feat.pkl', 'rb') as file:
    train_feat, train_lab = pickle.load(file)

with open('/Users/alejandro/AnacondaProjects/dev_set_new_feat.pkl', 'rb') as file:
    dev_feat, dev_lab = pickle.load(file)

In [35]:
model = LinearSVC()
model.fit(train_feat, train_lab)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [36]:
pred = model.predict(train_feat)
in_sample_acc = accuracy_score(pred, train_lab)
print('In sample accuracy:', in_sample_acc)
pred[:5]

In sample accuracy: 0.983


array([0, 1, 0, 1, 0])

In [37]:
pred = model.predict(dev_feat)
out_sample_acc = accuracy_score(pred, dev_lab)
print('Out of sample accuracy:', out_sample_acc)

Out of sample accuracy: 0.977777777778


In [38]:
# Save model for future retrieval
joblib.dump(model, "/Users/alejandro/AnacondaProjects/cv/svm_model.pkl", protocol=2)
print("Model saved to: /Users/alejandro/AnacondaProjects/cv/svm_model.pkl")

Model saved to: /Users/alejandro/AnacondaProjects/cv/svm_model.pkl


In [39]:
import cv2
import numpy as np
from keras.models import load_model

model_cnn = load_model('/Users/alejandro/AnacondaProjects/cv/model_cnn.h5')

img1 = cv2.imread('/Users/alejandro/AnacondaProjects/Hand_Data/test/hand/frame_flip_35.jpg', 0).reshape(1, 100, 80, 1)
img2 = cv2.imread('/Users/alejandro/AnacondaProjects/Hand_Data/test/no_hand/frame_flip_89.jpg', 0).reshape(1, 100, 80, 1)
img = np.concatenate((img1, img2))

feat = model_cnn.predict(img)
feat = np.concatenate((feat, train_feat[30].reshape(1, -1)))
model.predict(feat)

/anaconda3/envs/cv/lib/python3.6/site-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


array([1, 1, 0])

In [51]:
def pred_from_file(files, model_cnn, model, color=0):
    predictions = []
    with progressbar.ProgressBar(max_value=len(files)) as bar:
        for i in range(len(files)):
            img = cv2.imread(path + files[i], color)
            img = img.reshape((1, 100, 80, 1))//255
            feat = model_cnn.predict(img)
            pred = model.predict(feat)
            predictions.append(pred)
            bar.update(i)
    return predictions

In [54]:
import os
import progressbar

# Out-of-sample performance
path = '/Users/alejandro/AnacondaProjects/Hand_Data/test/hand/'
files = os.listdir(path)
pred_hand = pred_from_file(files, model_cnn, model, color=0)
path = '/Users/alejandro/AnacondaProjects/Hand_Data/test/no_hand/'
files = os.listdir(path)
pred_no_hand = pred_from_file(files, model_cnn, model, color=0)
FN = round(np.sum(pred_hand)/len(files), 4) * 100
FP = 100 - round(np.sum(pred_no_hand)/len(files), 4) * 100
TN = 100 - FN
TP = 100 - FP
Acc = (TP + TN) / (TP + TN + FP + FN)

print('Test sample performance:')
print('False negatives rate:', FN)
print('False positive rate:', FP)
print('Accuracy:', Acc)

100% (981 of 981) |#######################| Elapsed Time: 0:00:04 Time: 0:00:04
100% (1192 of 1192) |#####################| Elapsed Time: 0:00:05 Time: 0:00:05


In-sample performance:
False negatives rate: 1.76
False positive rate: 1.93
Accuracy: 0.98155
